<a href="https://colab.research.google.com/github/WaliMuhammadAhmad/CODEX-FINETUNNING/blob/main/codet5_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install --upgrade torch torchvision transformers datasets sentencepiece peft accelerate

In [ ]:
import os
import json
import torch
import shutil
import pandas as pd
import numpy as np
import transformers

In [ ]:
from datasets import load_dataset
from datasets import Dataset, load_dataset
from huggingface_hub import login, Repository
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    T5ForConditionalGeneration,
    RobertaTokenizer
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)

In [ ]:
from google.colab import userdata

In [ ]:
login(userdata.get('HF_TOKEN'),add_to_git_credential=True)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
base_model = "Salesforce/codet5-base"

model_path = "peft_model"

tokenizer_path = "tokenizer"

dataset = "CodexAI/Eval4Deepseek-Coder"  # dataset name at huggingface

dataset_dir = "dataset"  # dataset dir path

repo_url = f'https://huggingface.co/datasets/{dataset}'

In [ ]:
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

In [ ]:
repo = Repository(local_dir=dataset_dir,clone_from=repo_url)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/kaggle/working/dataset is already a clone of https://huggingface.co/datasets/CodexAI/Eval4Deepseek-Coder. Make sure you pull the latest changes with `repo.git_pull()`.


# Playing with Dataset

In [ ]:
def load_json_data(dir_name):
  """
  This function is used to load the json data from the given directory.
  After reading the data store them in a list
  After storing all the data in a list we can return the list.
  """

  data=[]
  for root_folder in os.listdir(dir_name):
    if root_folder!=".git" and root_folder!=".gitattributes":
      for files in os.listdir(os.path.join(dir_name,root_folder)):
        if files.endswith(".json"):
          with open(os.path.join(dir_name,root_folder,files),"r")as f:
            json_file=json.load(f)
            data.append(json_file)
  return data

In [ ]:
json_data=load_json_data(dataset_dir)

In [ ]:
json_data=json_data[:40000]

In [ ]:
json_data[0]['instruction']

'Generate a unit test case for the following Java method: SHA { public static byte[] encryptSHA(byte[] data) throws Exception { MessageDigest sha = MessageDigest.getInstance(KEY_SHA1); sha.update(data); return sha.digest(); } SHA(); static byte[] encryptSHA(byte[] data); static String encryptSHA(String data); static String encryptSHA(String str,String ecryptCode); static void main(String[] args);  }'

In [ ]:
json_data[0]['output']

'The unit test case for the given Java method is: @Test public void testEncryptSHAStringString() throws NoSuchAlgorithmException { Assert.assertEquals("", SHA.encryptSHA(null, null)); Assert.assertEquals(SHA_foo, SHA.encryptSHA("foo", "SHA")); Assert.assertEquals(SHA_1_foo, SHA.encryptSHA("foo", "SHA-1")); Assert.assertEquals(SHA_256_foo, SHA.encryptSHA("foo", "SHA-256")); Assert.assertEquals(SHA_bar, SHA.encryptSHA("bar", "SHA")); Assert.assertEquals(SHA_1_bar, SHA.encryptSHA("bar", "SHA-1")); Assert.assertEquals(SHA_256_bar, SHA.encryptSHA("bar", "SHA-256")); }'

In [ ]:
data=Dataset.from_list(json_data)

In [ ]:
data

Dataset({
    features: ['instruction', 'output'],
    num_rows: 40000
})

In [ ]:
data=data.train_test_split(test_size=0.2)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 32000
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 8000
    })
})

In [ ]:
train=data['train']
test=data['test']

In [ ]:
train

Dataset({
    features: ['instruction', 'output'],
    num_rows: 32000
})

In [ ]:
test

Dataset({
    features: ['instruction', 'output'],
    num_rows: 8000
})

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokens=tokenizer.tokenize(train['instruction'][0])
print("Token:\n",tokens)

Token:
 ['Generate', 'Ġa', 'Ġunit', 'Ġtest', 'Ġcase', 'Ġfor', 'Ġthe', 'Ġfollowing', 'ĠJava', 'Ġmethod', ':', 'ĠEmail', 'Output', 'Ġimplements', 'ĠReport', 'OutputHandler', 'Ġ{', 'Ġpublic', 'Ġint', 'Ġgenerate', '(', 'Ġfinal', 'ĠMaster', 'Report', 'Ġreport', ',', 'Ġfinal', 'Ġint', 'Ġaccepted', 'Page', ',', 'Ġfinal', 'ĠOutputStream', 'ĠoutputStream', ',', 'Ġfinal', 'Ġint', 'Ġyield', 'Rate', 'Ġ)', 'Ġthrows', 'ĠReport', 'ProcessingException', ',', 'ĠIOException', ',', 'ĠContent', 'IOException', 'Ġ{', 'Ġfinal', 'ĠI', 'ApplicationContext', 'Ġctx', 'Ġ=', 'ĠP', 'ent', 'ah', 'o', 'System', '.', 'get', 'ApplicationContext', '();', 'Ġif', 'Ġ(', 'Ġctx', 'Ġ==', 'Ġnull', 'Ġ)', 'Ġ{', 'Ġreturn', 'Ġ-', '1', ';', 'Ġ}', 'Ġtry', 'Ġ{', 'Ġfinal', 'ĠConfiguration', 'Ġconfiguration', 'Ġ=', 'Ġreport', '.', 'get', 'Configuration', '();', 'Ġif', 'Ġ(', 'Ġconfiguration', 'Ġinstanceof', 'ĠMod', 'ifiable', 'Configuration', 'Ġ)', 'Ġ{', 'Ġfinal', 'ĠMod', 'ifiable', 'Configuration', 'Ġmod', 'ifiable', 'Configuration', '

In [ ]:
token_id=tokenizer.convert_tokens_to_ids(tokens)
print("Token id:\n",token_id)

Token id:
 [4625, 279, 2836, 1842, 648, 364, 326, 3751, 5110, 707, 30, 8166, 1447, 4792, 8706, 30796, 288, 1071, 509, 2103, 12, 727, 13453, 4820, 2605, 16, 727, 509, 8494, 1964, 16, 727, 8962, 14956, 16, 727, 509, 2824, 4727, 262, 1216, 8706, 23684, 16, 1860, 16, 3697, 14106, 288, 727, 467, 28278, 1103, 273, 453, 319, 9795, 83, 3163, 18, 588, 28278, 5621, 309, 261, 1103, 422, 446, 262, 288, 327, 300, 21, 31, 289, 775, 288, 727, 4659, 1664, 273, 2605, 18, 588, 1750, 5621, 309, 261, 1664, 1276, 3431, 8424, 1750, 262, 288, 727, 3431, 8424, 1750, 681, 8424, 1750, 273, 261, 1739, 8424, 1750, 13, 1664, 31, 681, 8424, 1750, 18, 542, 809, 1396, 12, 5430, 1388, 3120, 18, 706, 5997, 67, 15066, 16, 315, 3767, 6, 11272, 681, 8424, 1750, 18, 542, 809, 1396, 12, 5430, 1388, 3120, 18, 2294, 11702, 15641, 67, 15066, 16, 315, 5743, 6, 11272, 681, 8424, 1750, 18, 542, 809, 1396, 12, 534, 1974, 6344, 67, 5151, 67, 3033, 67, 13603, 55, 16, 315, 3767, 6, 11272, 289, 727, 6183, 3458, 273, 394, 6183, 5621, 7

In [ ]:
original_tokens=tokenizer.convert_ids_to_tokens(token_id)
print("Original token:\n",original_tokens)

Original token:
 ['Generate', 'Ġa', 'Ġunit', 'Ġtest', 'Ġcase', 'Ġfor', 'Ġthe', 'Ġfollowing', 'ĠJava', 'Ġmethod', ':', 'ĠEmail', 'Output', 'Ġimplements', 'ĠReport', 'OutputHandler', 'Ġ{', 'Ġpublic', 'Ġint', 'Ġgenerate', '(', 'Ġfinal', 'ĠMaster', 'Report', 'Ġreport', ',', 'Ġfinal', 'Ġint', 'Ġaccepted', 'Page', ',', 'Ġfinal', 'ĠOutputStream', 'ĠoutputStream', ',', 'Ġfinal', 'Ġint', 'Ġyield', 'Rate', 'Ġ)', 'Ġthrows', 'ĠReport', 'ProcessingException', ',', 'ĠIOException', ',', 'ĠContent', 'IOException', 'Ġ{', 'Ġfinal', 'ĠI', 'ApplicationContext', 'Ġctx', 'Ġ=', 'ĠP', 'ent', 'ah', 'o', 'System', '.', 'get', 'ApplicationContext', '();', 'Ġif', 'Ġ(', 'Ġctx', 'Ġ==', 'Ġnull', 'Ġ)', 'Ġ{', 'Ġreturn', 'Ġ-', '1', ';', 'Ġ}', 'Ġtry', 'Ġ{', 'Ġfinal', 'ĠConfiguration', 'Ġconfiguration', 'Ġ=', 'Ġreport', '.', 'get', 'Configuration', '();', 'Ġif', 'Ġ(', 'Ġconfiguration', 'Ġinstanceof', 'ĠMod', 'ifiable', 'Configuration', 'Ġ)', 'Ġ{', 'Ġfinal', 'ĠMod', 'ifiable', 'Configuration', 'Ġmod', 'ifiable', 'Configur

In [ ]:
tokenizer(train['instruction'][0],max_length=100,truncation=True,padding="max_length",return_tensors="pt")

{'input_ids': tensor([[    1,  4625,   279,  2836,  1842,   648,   364,   326,  3751,  5110,
           707,    30,  8166,  1447,  4792,  8706, 30796,   288,  1071,   509,
          2103,    12,   727, 13453,  4820,  2605,    16,   727,   509,  8494,
          1964,    16,   727,  8962, 14956,    16,   727,   509,  2824,  4727,
           262,  1216,  8706, 23684,    16,  1860,    16,  3697, 14106,   288,
           727,   467, 28278,  1103,   273,   453,   319,  9795,    83,  3163,
            18,   588, 28278,  5621,   309,   261,  1103,   422,   446,   262,
           288,   327,   300,    21,    31,   289,   775,   288,   727,  4659,
          1664,   273,  2605,    18,   588,  1750,  5621,   309,   261,  1664,
          1276,  3431,  8424,  1750,   262,   288,   727,  3431,  8424,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1

In [ ]:
def tokenize_data(data):
  input_col=tokenizer(data['instruction'],max_length=512,truncation=True,padding="max_length",return_tensors="pt")
  target_col=tokenizer(data['output'],max_length=512,truncation=True,padding="max_length",return_tensors="pt")

  return {
      "input_ids":input_col["input_ids"],
      "attention_mask":input_col["attention_mask"],
      "labels":target_col["input_ids"]
  }

In [ ]:
print("Mapping train data...")
train=train.map(tokenize_data,batched=True)

Mapping train data


Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

In [ ]:
print("Mappig test data...")
test=test.map(tokenize_data,batched=True)

Mappig test data


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
train

Dataset({
    features: ['instruction', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 32000
})

In [ ]:
train=train.remove_columns(["instruction","output"])
test=test.remove_columns(["instruction","output"])

In [ ]:
train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 32000
})

In [ ]:
test

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8000
})

# Fine-tunning

In [ ]:
if torch.cuda.is_available():
  device="cuda"
  print(f"CUDA device: {torch.cuda.get_device_name()}")

CUDA device: Tesla P100-PCIE-16GB


In [ ]:
torch_type=torch.bfloat16

In [ ]:
peft_config=LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q","v"]
)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base',device_map={"": 0})

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
peft_model=get_peft_model(model,peft_config)

In [ ]:
print(f"BF16 support is {transformers.file_utils.is_torch_bf16_available()}")

BF16 support is True


/opt/conda/lib/python3.10/site-packages/transformers/utils/import_utils.py:455: FutureWarning: The util is_torch_bf16_available is deprecated, please use is_torch_bf16_gpu_available or is_torch_bf16_cpu_available instead according to whether it's used with cpu or gpu
  warnings.warn(


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    save_steps=500,
    num_train_epochs=1,
    fp16=False ,
    bf16=True,
    gradient_accumulation_steps=4
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer=Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Error: You must call wandb.init() before wandb.log()

In [ ]:
peft_model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

# Evaluation

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)

In [ ]:
code_eval = load("code_eval")  # or replace with the appropriate metric

In [ ]:
def evaluate_model(model, tokenizer, test_data):
    results = []
    for example in test_data:
        # Tokenize input and generate output
        inputs = tokenizer(example['instruction'], return_tensors="pt", truncation=True, padding="max_length", max_length=512)
        outputs = model.generate(**inputs, max_length=512, num_beams=5, do_sample=True)
        generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Evaluate the generated code
        metric_results = code_eval.compute(predictions=[generated_code], references=[example['reference_code']])
        results.append(metric_results)

    return results

In [ ]:
test_data = [
    {"instruction": example['instruction'], "reference_code": example['output']}
    for example in test
]

In [ ]:
evaluation_results = evaluate_model(model, tokenizer, test_data)

for result in evaluation_results:
    print(result)

# Inference

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)

In [ ]:
def generate_unit_tests(instruction):

  inputs = tokenizer(instruction, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

  outputs = model.generate(
      input_ids=inputs["input_ids"],
      attention_mask=inputs["attention_mask"],
      max_length=512,
      num_beams=5,
      do_sample=True,  # Enable sampling for diverse output
      temperature=1,  # Control randomness
      top_k=50,  # Limit the sampling pool to top K tokens
      top_p=0.9,
      no_repeat_ngram_size=2,
      early_stopping=True
  )

  # Decode the generated output
  generated_test = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return generated_test

In [ ]:
code = """
public class SimpleCalculator {
    // Method to add two numbers
    public int add(int a, int b) {
        return a + b;
    }

    // Method to subtract two numbers
    public int subtract(int a, int b) {
        return a - b;
    }

    // Method to multiply two numbers
    public int multiply(int a, int b) {
        return a * b;
    }

    // Method to divide two numbers
    // Throws ArithmeticException if divisor is zero
    public double divide(int a, int b) {
        if (b == 0) {
            throw new ArithmeticException("Cannot divide by zero");
        }
        return (double) a / b;
    }
}
"""
prompt="Generate a unit test case for the following Java method: "+code

In [ ]:
generated_test = generate_unit_tests(prompt)
print(generated_test)